In [83]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from db_utils import *

def query_db(host,query, params):
    conn = pymysql.connect(host =host, read_default_file="/etc/mysql/conf.d/analytics-research-client.cnf")
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    cursor.execute(query, params)
    rows = cursor.fetchall()
    conn.close()
    return mysql_to_pandas(rows)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- [QuickSurveysResponses_15266417](https://meta.wikimedia.org/wiki/Schema:QuickSurveysResponses)
- [QuickSurveyInitiation_15278946](https://meta.wikimedia.org/wiki/Schema:QuickSurveyInitiation)
- [Survey](https://docs.google.com/spreadsheets/d/1JD8-knLmnFXVwXxJYx6w9RRmxZWasSLaSKvj85SgrzE/edit?ts=56c3ccd2#gid=1291145097)

In [84]:
response_table = 'log.QuickSurveysResponses_15266417'
impression_table = 'log.QuickSurveyInitiation_15278946'
host = 'analytics-store.eqiad.wmnet'

### EL Summary Stats

In [99]:
query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveyInitiation_15278946
WHERE
    event_eventName ='eligible'
"""

n_eligible = query_db(host, query, {}).iloc[0]['n']

query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveyInitiation_15278946
WHERE
    event_eventName ='impression'
"""

n_impressions = query_db(host, query, {}).iloc[0]['n']

query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveysResponses_15266417
"""
n_responses = query_db(host, query, {}).iloc[0]['n']

query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveysResponses_15266417
WHERE
    event_surveyResponseValue ='ext-quicksurveys-external-survey-yes-button'
"""
n_yes = query_db(host, query, {}).iloc[0]['n']

query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveysResponses_15266417
WHERE
    event_surveyResponseValue ='ext-quicksurveys-external-survey-no-button'
"""
n_no = query_db(host, query, {}).iloc[0]['n']



print('Eligible Pageviews: ', n_eligible)
print('Widget Impressions: ', n_impressions)
print('Clicks: ', n_responses)
print('Yeses: ', n_yes)
print('Nos: ', n_no)

Eligible Pageviews:  15574
Widget Impressions:  12424
Clicks:  149
Yeses:  37
Nos:  112


### Does each eligible pageview have a unique survey token?

In [100]:
query = """
SELECT COUNT(DISTINCT(event_surveyInstanceToken)) as n
FROM log.QuickSurveyInitiation_15278946
WHERE event_eventName ='eligible'
"""
print(query_db(host,query, {}).iloc[0]['n'])

15572


Pretty Close

### Does each widget impression have a unique survey token?

In [101]:
query = """
SELECT COUNT(DISTINCT(event_surveyInstanceToken)) as n
FROM log.QuickSurveyInitiation_15278946
WHERE event_eventName ='impression'
"""
print(query_db(host,query, {}).iloc[0]['n'])

12421


Pretty Close.

### Does each Widget Impression have a matching Pageview?

In [103]:
query = """
SELECT
    COUNT(DISTINCT(i.event_surveyInstanceToken)) as n
FROM
    (SELECT event_surveyInstanceToken
    FROM log.QuickSurveyInitiation_15278946
    WHERE
    event_eventName ='impression') i
JOIN
    (SELECT event_surveyInstanceToken
    FROM log.QuickSurveyInitiation_15278946
    WHERE
    event_eventName ='eligible') e
ON
    (i.event_surveyInstanceToken = e.event_surveyInstanceToken)
"""

df = query_db(host,query, {})
print(df.iloc[0]['n'])

12405


Close. It seems there are a small number of impressions without a matching pageview.

### Does each click have a matching Widget Impression?

In [104]:
query = """
SELECT *
FROM
    log.QuickSurveysResponses_15266417 r,
    log.QuickSurveyInitiation_15278946 i
WHERE
    r.event_surveyInstanceToken = i.event_surveyInstanceToken
    AND i.event_eventName ='impression'
""" 

d_click = query_db(host, query, {})
print(d_click.shape[0], len(set(d_click['event_surveyInstanceToken'])))

148 148


Each click has exactly one widget impression. One click without a matching rendered widget

### Does each Google Survey Response have a registered click

In [105]:
d_survey = pd.read_csv('responses.tsv', sep = '\t')
st = 'This is you survey ID. Please do not modify.'
ct = 'event_surveyInstanceToken'
df_survey_click = d_survey.merge(d_click, how = 'inner', right_on = ct, left_on = st)

In [106]:
print(df_survey_click.shape[0])

23


Only 23 out of 33 survey responses have a matching click.

### Does each Google Survey Response have a registered impression?

In [108]:
query = """
SELECT
    event_surveyInstanceToken
FROM log.QuickSurveyInitiation_15278946
WHERE
    event_eventName ='impression'
"""

d_impression = query_db(host,query, {})
df_survey_impression = d_survey.merge(d_impression, how = 'inner', right_on = 'event_surveyInstanceToken', left_on = st)

In [109]:
print(df_survey_impression.shape[0])

24


Only 24 out of 33 survey responses have a matching click.

### Summary

In [110]:
print('Eligible Pageviews: ', n_eligible)
print('Widget Impressions: ', n_impressions)
print('Clicks: ', n_responses)
print('Yeses: ', n_yes)
print('Google Responses tracked in EL', df_survey_click.shape[0])
print('Google Responses', d_survey.shape[0])

Eligible Pageviews:  15574
Widget Impressions:  12424
Clicks:  149
Yeses:  37
Google Responses tracked in EL 23
Google Responses 33


- Don't know how many users where in the sample
- One in 300 pageviews results in a 'Yes'
- 62% of 'Yes' events lead to a survery response
- only 60% of survey responses are trackable in EL